In [136]:
%load_ext autoreload
%autoreload 2

from processLIDC import Patient
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
patient_id = 'LIDC-IDRI-0002'
patient = Patient(patient_id)
patient.scale()

In [138]:
patient.get_all_nodules()

Paciente: LIDC-IDRI-0002
___________________________________
Paciente del nodulo: LIDC-IDRI-0002
Primera slice con el nodulo 171
num. slices nodulo: 28
-----------
Paciente del nodulo: LIDC-IDRI-0002
Primera slice con el nodulo 176
num. slices nodulo: 23
-----------
___________________________________


In [139]:
# patient.reconstruct_body(nodulos=True)

In [140]:
# patient.imshow(slices=(172,173,174,175,176,177,178,179), label=True, scaled=True)

## Funcion para guardar las imagenes en formato .jpg

In [141]:
def dicom2jpg(id_patient, path2newdataset = './dataset'):
    patient = Patient(id_patient)
    patient.scale()
    h, w, n_slices = np.shape(patient.imgs_scaled)
    for i in range(n_slices):
        fullpath = f'{path2newdataset}/images/{id_patient}_{i}.jpg'
        imagen_8bits = np.interp(patient.imgs_scaled[:,:,i], (0, 8), (0, 255)).astype(np.uint8)
        cv2.imwrite(fullpath, imagen_8bits)


def ann2yolo(id_patient, path2newdataset = './dataset'):
    patient = Patient(id_patient)
    # patient.scale()
    h, w, n_slices = np.shape(patient.mask)
    # print(n_slices)
    no_tumor = 0
    tumor =0
    for i in range(n_slices):
        fullpath = f'{path2newdataset}/labels/{id_patient}_{i}.txt'
        mask = np.interp(patient.mask[:,:,i], (0, 1), (0, 255)).astype(np.uint8)
        _, threshold = cv2.threshold(mask, 100, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if np.shape(contours) ==(0,):
            no_tumor+=1
        else:
            tumor+=1
        # Crear un archivo de texto para escribir los contornos
        with open(fullpath, "w") as file:
            for contour in contours:
                # Escribir '0' al principio de cada línea
                file.write("0")

                # Obtener las coordenadas de los puntos del contorno
                points = contour.reshape(-1, 2)

                # Escribir las coordenadas en el archivo
                for point in points:
                    x, y = point
                    file.write(f" {x/w} {y/h}")

                # Nueva línea para el siguiente contorno
                file.write("\n")
    return tumor, no_tumor


def create_dataset(path2olddataset,path2newdataset, val = 0.2):
    id_patients = os.listdir(path2olddataset)
    archivo = open('./failed_patients.txt', 'r')  # Reemplaza 'nombre_archivo.txt' por el nombre de tu archivo

    failed_patients = []

    for linea in archivo:
        linea = linea.strip()  # Elimina los espacios en blanco al principio y al final de la línea
        failed_patients.append(linea)
    archivo.close()
    id_patients = [id_patient for id_patient in id_patients if id_patient not in failed_patients]
    tumor_total, no_tumor_total = 0,0
    for id_patient in tqdm(id_patients):
        # try:
        # print(id_patient)
        train_random = random.random() > val
        # print(train_random)
        if train_random:
            dicom2jpg(id_patient, path2newdataset+'/train')
            tumor, no_tumor = ann2yolo(id_patient, path2newdataset+'/train')
            tumor_total += tumor
            no_tumor_total += no_tumor
        else:
            dicom2jpg(id_patient, path2newdataset+'/validation')
            tumor, no_tumor = ann2yolo(id_patient, path2newdataset+'/validation')
            tumor_total += tumor
            no_tumor_total += no_tumor
        print('train:', len(os.listdir(path2newdataset+'/train/images')), len(os.listdir(path2newdataset+'/train/labels')))
        print('validation:', len(os.listdir(path2newdataset+'/validation/images')), len(os.listdir(path2newdataset+'/validation/labels')))
        # except:
#         #     print(id_patient, 'da fallos')
#         
    return tumor_total, no_tumor_total

In [142]:
path2newdataset='/home/faraujo/TFM/dataset_vacio (copy)'
# dicom2jpg(patient_id, path2newdataset)


path2olddataset = '/home/faraujo/TFM/manifest-1675801116903/LIDC-IDRI'

create_dataset(path2olddataset, path2newdataset, val=0.3)



  0%|          | 0/3 [00:00<?, ?it/s]

LIDC-IDRI-0002
True


 33%|███▎      | 1/3 [00:08<00:17,  8.98s/it]

train: 261 261
validation: 0 0
LIDC-IDRI-0005
True


 67%|██████▋   | 2/3 [00:14<00:06,  6.90s/it]

train: 394 394
validation: 0 0
LIDC-IDRI-0013
False


100%|██████████| 3/3 [00:20<00:00,  6.84s/it]

train: 394 394
validation: 125 125


(51, 468)